# Proof of Concept - Rolling Codes in FOB for Educational Purpose

## Idea
The idea was to build a simple rolling codes implementation in python to learn and demonstrate how a remote device (e.g. for car unlocking) works. 

## Notes
It is no secure or working implementation for real world use cases. I just did some assumptions based on the mechanics on BMWs from the E-Series and did some lose implementation of the KeeLoq system.
I also reduced complexity at some steps to make the core concepts more understandable. E.g. I implemented all data saving and transmission as JSON payloads and not, like in real world devices, as bits.

## How it works


## Use cases
1. Pairing of two devices, to show how it works with multiple devices and different seeds
2. Sending a valid request (e.g. pressing the button on the FOB)
3. Sending a valid request with different counter (e.g. FOB was used out of reach for a couple times)
4. Sending an invalid request with different counter (e.g. FOB was used out of reach for many times)
5. Replaying the same code again (Replay Attack)
6. Rolljam attack (saving the code and sending it later)


## Resources
KeeLoq Documentation: https://ww1.microchip.com/downloads/en/AppNotes/91002a.pdf

In [1]:
from Crypto.Hash import SHA256

### Settings ###
DEBUG = True

In [524]:
class Receiver:
    def __init__(self):
        self.receiver = {}
        
    def clear_receiver(self):
        self.receiver = {}
        
    def pair_new_device(self, pairing_payload):
        if pairing_payload['Id'] not in self.receiver:
            self.receiver[pairing_payload['Id']] = {
                'Count' : 0,
                'Codes' : self.generate_rolling_code_list(0, 10, pairing_payload['Id'], pairing_payload['Seed']),
                'Seed' : pairing_payload['Seed']
            }
            if DEBUG: print('Device paired')
        return self.receiver

    def generate_rolling_code_list(self, start_code, list_len, sender_id, seed):
        rolling_codes = []
        for i in range (start_code, list_len):
            hash = SHA256.new()
            hash.update(
                (sender_id + seed + str(i)).encode('utf-8')
            )
            rolling_codes.append(
                hash.hexdigest()
            )
        return rolling_codes

    def check_sender_id_in_allowlist(self, sender_id):
        if sender_id in self.receiver:
            if DEBUG: print('Sender is paired with receiver')
            return True
        else:
            raise Exception("Invalid Sender Id") 

    def check_rolling_code(self, payload):
        sender_count = self.receiver[payload['Id']]['Count'] 
        while True:
            try:
                receiver_code = self.receiver[payload['Id']]['Codes'][sender_count]
            except:
                raise Exception("FOB out of sync") 
            sender_count += 1
            if payload['Code'] == receiver_code:
                if DEBUG: print('Code valid')
                return sender_count
            print('Counter out of sync. Increase counter')

    def set_new_receiver_code_count(self, sender_id, count):
        self.receiver[payload['Id']]['Count'] = count
        if DEBUG: print('Receiver code count adjusted')

    def validate_payload_adjust_codes(self, payload):
        self.check_sender_id_in_allowlist(
            payload['Id']
        )

        sender_count_new = self.check_rolling_code(
            payload
        )
        
        self.receiver[payload['Id']]['Codes'].extend(
            self.generate_rolling_code_list(
                self.receiver[payload['Id']]['Count'] + 10, 
                (sender_count_new + 10), 
                payload['Id'], 
                self.receiver[payload['Id']]['Seed']
            )
        )

        self.set_new_receiver_code_count(
            payload['Id'], 
            sender_count_new
        )
        
        if DEBUG: print('Extended code list')

In [525]:
class Sender:
    def __init__(self):
        fob = {}
        
    def set_fob_data(self, fob):
        self.fob = fob
        
    def create_pairing_payload(self):
        return {
            'Id' : self.fob['Id'],
            'Seed' : self.fob['Seed']
        }
        
    def create_send_payload(self):
        return {
            'Id' : self.fob['Id'],
            'Code' : self.fob['Codes'][self.fob['Counter']]
        }
    
    def generate_rolling_code_list(self, start_code, list_len, sender_id, seed):
        rolling_codes = []
        for i in range (start_code, list_len):  
            hash = SHA256.new()
            hash.update(
                (sender_id + seed + str(i)).encode('utf-8')
            )
            rolling_codes.append(
                hash.hexdigest()
            )
        return rolling_codes
    
    def higher_counter(self):
        self.fob['Counter'] += 1
        return self.fob
    
    def extend_rolling_code(self):
        self.fob['Codes'].extend(
            self.generate_rolling_code_list(
                self.fob['Counter'] + 10, 
                (self.fob['Counter'] + 1 + 10), 
                self.fob['Id'], 
                self.fob['Seed']
            )
        )

In [527]:
### Pairing of two FOBs ###

#Note: The payloads are for educational purpose in JSON format. Usually it will be saved in bits

### Initial State of the devices ###
def pair_devices():
    receiver = Receiver()
    fob_1 = Sender()
    fob_2 = Sender()

    fob_1.set_fob_data(
        {
            'Id' : '213u123z1281',
            'Seed' : '7827637huasz23',
            'Counter' : 0,
            'Codes' : fob_1.generate_rolling_code_list(0, 10, '213u123z1281', '7827637huasz23'),
        }
    )

    fob_2.set_fob_data(
        {
            'Id' : '78923491823s',
            'Seed' : '3217676128hsa12',
            'Counter' : 0,
            'Codes' : fob_2.generate_rolling_code_list(0, 10, '78923491823s', '3217676128hsa12'),
        }
    )


    ### Payload from FOB (always sender, never receives) ###
    pairing_payload_fob_1 = fob_1.create_pairing_payload()
    pairing_payload_fob_2 = fob_2.create_pairing_payload()

    ### Mock a pairing process ###

    #Usually you will start the pairing process by doing a combination of ignition state and button pressing
    print('Pairing started ...')

    #At least in BMW cars all paired devices get removed after starting the process
    print('Clear all paired devices ...')
    receiver.clear_receiver()

    #Usually you will send a payload by pressing multiple buttons the same time at the FOB
    for payload in [pairing_payload_fob_1, pairing_payload_fob_2]:
        receiver.pair_new_device(payload)
        print('Waiting 10 seconds for other pairing')

    #Pairing ends if no new device is paired in a short timeframe or after turning off ignition
    print('Pairing done through timeout or power off')
    
    return [receiver, fob_1, fob_2]
    
receiver, fob_1, fob_2 = pair_devices()

Pairing started ...
Clear all paired devices ...
Device paired
Waiting 10 seconds for other pairing
Device paired
Waiting 10 seconds for other pairing
Pairing done through timeout or power off


In [534]:
### Case: Sender sends valid code to receiver ###

#Pair devices for clean start
receiver, fob_1, fob_2 = pair_devices()

#FOB creates Payload
payload = fob_1.create_send_payload()

#Receiver validates Payload, higher count after success and generate a new code at end of the list
receiver.validate_payload_adjust_codes(payload)

#FOB extend a new code at end of the list
fob_1.extend_rolling_code()

#FOB highers counter after each transmission
fob_1.higher_counter()

print('Done')

Pairing started ...
Clear all paired devices ...
Device paired
Waiting 10 seconds for other pairing
Device paired
Waiting 10 seconds for other pairing
Pairing done through timeout or power off
Sender is paired with receiver
Code valid
Receiver code count adjusted
Extended code list
Done


In [535]:
### Case: Sender2 sends code then Sender1 sends code ###

#Pair devices for clean start
receiver, fob_1, fob_2 = pair_devices()

#FOB creates Payload
payload = fob_1.create_send_payload()

#Receiver validates Payload, higher count after success and generate a new code at end of the list
receiver.validate_payload_adjust_codes(payload)

#FOB extend a new code at end of the list
fob_1.extend_rolling_code()

#FOB highers counter after each transmission
fob_1.higher_counter()

#Now the same for fob_2
print(10*'-')
payload = fob_2.create_send_payload()
receiver.validate_payload_adjust_codes(payload)
fob_2.extend_rolling_code()
fob_2.higher_counter()

print('Done')

Pairing started ...
Clear all paired devices ...
Device paired
Waiting 10 seconds for other pairing
Device paired
Waiting 10 seconds for other pairing
Pairing done through timeout or power off
Sender is paired with receiver
Code valid
Receiver code count adjusted
Extended code list
----------
Sender is paired with receiver
Code valid
Receiver code count adjusted
Extended code list
Done


In [536]:
### Case: Sender sends code <10 away from last code (FOB out of reach) ###

#Pair devices for clean start
receiver, fob_1, fob_2 = pair_devices()

#FOB creates Payload
payload = fob_1.create_send_payload()

#FOB higher counter to mock multiple button pressing without range while receiver is on old counter
for i in range(0, 8):
    fob_1.higher_counter()
    fob_1.extend_rolling_code()
payload = fob_1.create_send_payload()

#Receiver validates Payload, higher count after success and generate a new code at end of the list
receiver.validate_payload_adjust_codes(payload)

#FOB extend a new code at end of the list
fob_1.extend_rolling_code()

#FOB highers counter after each transmission
fob_1.higher_counter()

print('Done')

Pairing started ...
Clear all paired devices ...
Device paired
Waiting 10 seconds for other pairing
Device paired
Waiting 10 seconds for other pairing
Pairing done through timeout or power off
Sender is paired with receiver
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Code valid
Receiver code count adjusted
Extended code list
Done


In [537]:
### Case: Sender sends code >10 away from last code (FOB out of reach) ###

#Pair devices for clean start
receiver, fob_1, fob_2 = pair_devices()

#FOB creates Payload
payload = fob_1.create_send_payload()

#FOB higher counter to mock multiple button pressing without range while receiver is on old counter
for i in range(0, 20):
    fob_1.higher_counter()
    fob_1.extend_rolling_code()
payload = fob_1.create_send_payload()

#Receiver validates Payload, higher count after success and generate a new code at end of the list
receiver.validate_payload_adjust_codes(payload)

#FOB extend a new code at end of the list
fob_1.extend_rolling_code()

#FOB highers counter after each transmission
fob_1.higher_counter()

print('Done')

Pairing started ...
Clear all paired devices ...
Device paired
Waiting 10 seconds for other pairing
Device paired
Waiting 10 seconds for other pairing
Pairing done through timeout or power off
Sender is paired with receiver
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter


Exception: FOB out of sync

In [538]:
### Case: Sender replays old code - Replay Attack ###

#Pair devices for clean start
receiver, fob_1, fob_2 = pair_devices()

#FOB creates Payload
payload = fob_1.create_send_payload()

payload_saved = payload

#Receiver validates Payload, higher count after success and generate a new code at end of the list
receiver.validate_payload_adjust_codes(payload)

#FOB extend a new code at end of the list
fob_1.extend_rolling_code()

#FOB highers counter after each transmission
fob_1.higher_counter()

#Replaying an already sent payload
receiver.validate_payload_adjust_codes(payload_saved)

#FOB lose sync because the payload is valid but not in sync with the increment counter

Pairing started ...
Clear all paired devices ...
Device paired
Waiting 10 seconds for other pairing
Device paired
Waiting 10 seconds for other pairing
Pairing done through timeout or power off
Sender is paired with receiver
Code valid
Receiver code count adjusted
Extended code list
Sender is paired with receiver
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter


Exception: FOB out of sync

In [539]:
### Case: RollJam Attack ###

#Pair devices for clean start
receiver, fob_1, fob_2 = pair_devices()

#FOB creates Payload
payload = fob_1.create_send_payload()

#An attacker now jams the transmission but captures the key
payload_saved_1 = payload

#The key still generates a new key and higher counter
fob_1.higher_counter()
fob_1.extend_rolling_code()

#The user sees that the car doesn't locks and press the button again. 
payload = fob_1.create_send_payload()

#The attacker now jams it again and captures it
payload_saved_2 = payload

#Now the attacker sends first captured payload to give the user a positive feedback (car locks)
#Receiver validates Payload, higher count after success and generate a new code at end of the list
receiver.validate_payload_adjust_codes(payload_saved_1)

#FOB extend a new code at end of the list
fob_1.extend_rolling_code()

#FOB highers counter after each transmission
fob_1.higher_counter()

#The attacker has now an unsed key which is still in sync with the counter in receiver (because jammed before)
#The attacker can send this payload at a later time and gains control
receiver.validate_payload_adjust_codes(payload_saved_2)

Pairing started ...
Clear all paired devices ...
Device paired
Waiting 10 seconds for other pairing
Device paired
Waiting 10 seconds for other pairing
Pairing done through timeout or power off
Sender is paired with receiver
Code valid
Receiver code count adjusted
Extended code list
Sender is paired with receiver
Code valid
Receiver code count adjusted
Extended code list
